In [1]:
import mph

In [2]:
client = mph.start()
pymodel = client.create('cavity')

In [3]:
parameters = pymodel/'parameters'
(parameters/'Parameters 1').rename('Parameters')

pymodel.parameter('cavity_width', '30[mm]')
pymodel.parameter('cavity_length', '7[mm]')
pymodel.parameter('cavity_height', '28[mm]')
pymodel.parameter('cavity_fillet', '2.5[mm]')
pymodel.parameter('port_distance', '12[mm]')
pymodel.parameter('port_height', '7[mm]')
pymodel.parameter('port_in_depth', '4[mm]')
pymodel.parameter('port_out_depth', '6.5[mm]')
pymodel.parameter('port_inner_radius', '0.3[mm]')
pymodel.parameter('port_outer_radius', '2.1[mm]')

pymodel.description('cavity_width', 'Cavity Width')
pymodel.description('cavity_length', 'Cavity Length')
pymodel.description('cavity_height', 'Cavity Height')
pymodel.description('cavity_fillet', 'Rounding Radius for cavity') 

In [ ]:
## Java like
# model = pymodel.java
# model.component().create("comp1", True)
# model.component("comp1").geom().create("geom1", 3) # 3 means 3D
# model.component("comp1").geom("geom1").lengthUnit("\u00b5m")
# model.component("comp1").geom("geom1").create("wp1", "WorkPlane")
# model.component("comp1").geom("geom1").feature("wp1").set("unite", True)
# model.component("comp1").geom("geom1").feature("wp1").geom().create("r1", "Rectangle")
# model.component("comp1").geom("geom1").feature("wp1").geom().feature("r1").set("size", [2.4, 0.7])
# model.component("comp1").geom("geom1").feature("wp1").geom().feature("r1").set("pos", [-1.4, -0.35])
# model.component("comp1").geom("geom1").feature("wp1").geom().run("r1")

#
model = pymodel.java
comp1 = model.component().create("comp1", True)

geom1 = comp1.geom().create("geom1", 3) # 3 means 3D
geom1.lengthUnit("um")
geom1.run()

# Work plane 1
wp1 = geom1.create("wp1", "WorkPlane")
wp1.set("unite", True)
r1 = wp1.geom().create("r1", "Rectangle")
r1.set("size", ['cavity_width', 'cavity_length'])
r1.set("pos", ['-cavity_width/2', '0'])
wp1.geom().run("r1")

fil1 = wp1.geom().create("fil1", "Fillet")
fil1.selection("pointinsketch").set("r1", 1, 2, 3, 4)
fil1.set("radius", 'cavity_fillet')
wp1.geom().run("fil1")

wp1.label("cav-def")
geom1.run("wp1")

# Extrude 1
ext1 = geom1.create("ext1", "Extrude")
ext1.set("workplane", "wp1")
ext1.selection("input").set("wp1")
ext1.setIndex("distance", 'cavity_height', 0) # 0 means first row ?

geom1.run("ext1")

# Work plane 2
wp2 = geom1.create("wp2", "WorkPlane")
wp2.set("unite", True)
wp2.label("Ports-def")
wp2.set("quickplane", "xz")
c1 = wp2.geom().create("c1", "Circle")
c1.set("r", 'port_outer_radius')
c1.set("pos", ['0', '(cavity_height + port_distance)/2'] ) # Both x, y values must be int/float or string

mir1 = wp2.geom().create("mir1", "Mirror")
mir1.selection("input").set("c1")
mir1.set("pos", ['0', 'cavity_height/2'])
mir1.set("axis", [0., 1.])
mir1.set("keep", True)
wp2.geom().run("mir1")
geom1.run('wp2')

# Extrude 2

ext2 = geom1.create("ext2", "Extrude")
ext2.set("workplane", "wp2")
ext2.selection("input").set("wp2")
ext2.setIndex("distance", "port_height", 0)
ext2.label("Ports")

geom1.run("ext2")

# Work plane 3
wp3 = geom1.create("wp3", "WorkPlane")
wp3.set("unite", True)
wp3.set("quickplane", "xz")
wp3.set("quicky", "-port_height")
c2 = wp3.geom().create("c2", "Circle")
c2.set("pos",  ['0', '(cavity_height + port_distance)/2'] )
c2.set("r", 'port_inner_radius')
wp3.label("Pin-def")
geom1.run('wp3')

# Extrude 3
ext3 = geom1.create("ext3", "Extrude")
ext3.set("workplane", "wp3")
ext3.selection("input").set("wp3")
ext3.setIndex("distance", "-port_in_depth", 0)
ext3.label("Pin")
geom1.run("ext3")

# Work plane 4
wp4 = geom1.create("wp4", "WorkPlane")
wp4.set("unite", True)
wp4.set("quickplane", "xz")
wp4.set("quicky", "-port_height")
c3 = wp4.geom().create("c2", "Circle")
c3.set("pos",  ['0', '(cavity_height - port_distance)/2'] )
c3.set("r", 'port_inner_radius')
wp4.label("Pout-def")
geom1.run('wp4')

# Extrude 4
ext4 = geom1.create("ext4", "Extrude")
ext4.set("workplane", "wp4")
ext4.selection("input").set("wp4")
ext4.setIndex("distance", "-port_out_depth", 0)
ext4.label("Pout")
geom1.run("ext4")

# Set view after making everything
comp1.view("view1").set("transparency", True)

<java object 'com.comsol.clientapi.impl.ViewClient'>

In [6]:
pymodel.save('mph/cavity_demo.mph')
client.remove(pymodel)